In [ ]:
## function
# package - pymupdf - pytesseract
# install PyMuPDF และ pytesseract
!pip install pymupdf pytesseract pdf2image pillow

# install tesseract-ocr engine
# use Debian package
!apt-get install -y tesseract-ocr # -y tesseract-ocr ใช้ทำ OCR
!apt-get install -y tesseract-ocr-tha tesseract-ocr-eng #ภาษาไทย+อิ้งได้

# เชื่อม  gg drive
from google.colab import drive

# mount() ใช้เชื่อมต่อ Google Drive เข้ากับโค้ด
drive.mount('/content/drive')

import os


# list file in folder
print(os.listdir('/content/drive/MyDrive/OCR_test'))


def ocr_pdf_return_text(pdf_path,   # ระบุตำแหน่ง file pdf
                        dpi=300,    # คุณภาพของรูปภาพ
                        ocr_language='tha+eng'):
    import fitz  # PyMuPDF
    from pdf2image import convert_from_path

    import pytesseract
    from PIL import Image

    # ดูจน.หน้าใน pdf

    # fitz.oprn() เปิดไฟล์ pdf

    doc = fitz.open(pdf_path)
    print(f"Total pages: {doc.page_count}")

    # สร้างตัวแปร full_text
    full_text = ""

    # วนลูปทำงานทีละหน้า pdf
    for page_num in range(doc.page_count):
        page = doc[page_num] # หน้าปัจจุบัน
        text = page.get_text("text")  # ดึง text ถ้ามีแบบ text-based

        print(f"\nPage {page_num+1}/{doc.page_count}")

        '''
        ดูว่าข้อความที่ดึงได้ในหน้ามีมากกว่า 100 ตัวไหม -> ดึงจาก pdf ได้เลย
        ถ้าใช่มาต่อ full text ถ้าไม่ใช่แสดงว่าเป็นรูปภาพ  -> ใช้ OCR แปลงเป็นข้อความ
        '''

        if len(text.strip()) > 100:
            print("Detected as Text-based → Extracting text")
            full_text += f"\n\n--- Page {page_num+1} (Text-based) ---\n{text}"

        else:
            print("Detected as Image-based → Running OCR")
            images = convert_from_path(pdf_path,
                                       dpi=dpi,
                                       first_page=page_num+1,
                                       last_page=page_num+1)

            image = images[0] # แปลงทีละหน้า มีรูปภาพเดียวใน image

            ''' pytesseract ช่วยแปลงรูปภาพให้ได้เป็นข้อความ (ใข้ไทย-อิ้งที่เซ็ตไว้แล้วก่อนหน้า)
                แล้วเอาขคที่ได้จาก OCR มาต่อ full text
            '''

            ocr_text = pytesseract.image_to_string(image, lang=ocr_language)
            full_text += f"\n\n--- Page {page_num+1} (OCR) ---\n{ocr_text}"

    # คืนค่า full_text ที่เป็นข้อความที่ดึงได้จากไฟล์ PDF ทั้งหมด
    return full_text


def clean_ocr_text(raw_text):
    cleaned_lines = []  # สร้าง list เปล่า

    # วนลูปแยกข้อความเป็นบรรทัด
    for line in raw_text.splitlines():  # splitlines() ใช้แยกข้อความเป็นบรรทัด + วนลูปตรวจสอบแต่ละบรรทัด
        line = line.strip()             # strip() ลบเว้นวรรค ที่อยู่ต้นและท้ายบรรทัด

        if not line:                    # ถ้าเป็นบรรทัดว่าง -> ข้ามไปบรรทัดถัดไป
            continue

        if re.match(r"^-{3,}$", line):  # re.match() ดูว่ามีขีดคั่น (-) ตั้งแต่ 3 ตัวขึ้นไปไหม
            continue

        # เพิ่มบรรทัดที่ผ่านเงื่อนไขทั้งหมดไปที่list cleaned_lines
        cleaned_lines.append(line)

    # รวมบรรทัดทั้งหมดที่clean แล้ว มาเป็นขค.เดียว
    return "\n".join(cleaned_lines)


# clean ขค
def clean_text(text):
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)        # ลบช่องว่างระหว่างตัวเลข
    text = re.sub(r'([A-Z])\s+(\d)', r'\1\2', text)     # ลบช่องว่างระหว่างตัวอักษร A-Z กับตัวเลข
    text = re.sub(r'(\d)\s+([A-Z])', r'\1\2', text)     # ลบช่องว่างระหว่างตัวเลขกับตัวอักษร A-Z
    text = text.strip()                                 # ลบช่องว่างต้นและท้ายข้อความ
    return text

# ค้นหาและดึงข้อมูลสำคัญจากข้อความ ใช้ Regular Expressions (Regex)
# สร้าง dict เก็บขคที่ดึงมาได้
def extract_fields_from_text(text):
    result = {}

    # re.search(pattern, text, re.IGNORECASE) ใช้ค้นหา รูปแบบข้อมูลที่ต้องการ ตาม Regex Pattern
    study_match = re.search(r"Study\s*:\s*(.+)", text, re.IGNORECASE)
    visit_match = re.search(r"Visit\s*:\s*(.+)", text, re.IGNORECASE)

    # PID
    pid_match = re.search(r"P(?:ID|IO|LD|OD)\s*:\s*(\d+)", text, re.IGNORECASE)

    # Fixed ID
    fixed_id_match = re.search(r"F[^\n:]*?(?:ID|lD|Id|iD)[^\n]*?:\s*([0-9\-]+)", text, re.IGNORECASE)

    initials_match = re.search(r"Initials\s*:\s*([A-Z]+)", text, re.IGNORECASE)

    # group(1) ดึงขคที่เจอจาก regex
    result["Study"] = clean_text(study_match.group(1)) if study_match else None
    result["Visit"] = clean_text(visit_match.group(1)) if visit_match else None
    result["PID"] = int(pid_match.group(1)) if pid_match else None
    result["Fixed ID"] = fixed_id_match.group(1).strip() if fixed_id_match else None
    result["Initials"] = initials_match.group(1).strip() if initials_match else None

    return result


import re
import json


text = ocr_pdf_return_text("/content/drive/MyDrive/OCR_test/AdobeScan_10Jun2568.pdf",
                           dpi=300,
                           ocr_language='tha+eng')

cleaned_text = clean_ocr_text(text)

# OCR
text = ocr_pdf_return_text("/content/drive/MyDrive/OCR_test/AdobeScan_10Jun2568.pdf",
                           dpi=300,
                           ocr_language='tha+eng')

# Clean text ลบช่องว่างต่างๆ
cleaned_text = clean_ocr_text(text)

# Extract fields เอาขคที่อยากได้ออกมา
output_dict = extract_fields_from_text(cleaned_text)

# Show result as dict
print(json.dumps(output_dict,
                 indent=4,                  # จัดรูปให้มีระยะห่าง 4 ช่อง
                 ensure_ascii=False))       # ไม่ให้ภาษาไทยแปลงเป็น unicode


with open("output_result.txt", "w", encoding="utf-8") as f:
    for key, value in output_dict.items():
        f.write(f"{key}: {value}\n")


from google.colab import files
files.download("output_result.txt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 37.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
The following NEW packages will be installed:
  tesseract-ocr-tha
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 899 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-tha all 1:4.00~git30-7274cfa-1.1 [899 kB]
Fetched 899 kB in 1s (942 kB/s)
Selecting previously unselected package tesseract-ocr-tha.
(Reading database ... 126111 files and d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>